In [2]:
print("Hello World")

Hello World


In [7]:
%pip install kagglehub --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
  Using cached kagglehub-0.3.10-py3-none-any.whl.metadata (31 kB)
Using cached kagglehub-0.3.10-py3-none-any.whl (63 kB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("emmarex/plantdisease")

print("Path to dataset files:", path)

Resuming download from 116391936 bytes (573120754 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/emmarex/plantdisease?dataset_version_number=1 (116391936/689512690) bytes left.


100%|██████████| 658M/658M [05:00<00:00, 1.91MB/s]

Extracting files...


Path to dataset files: /home/peeranat/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1


In [14]:
%pip install opencv-python tensorflow keras numpy --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.9 MB/s eta 0:00:00
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (63.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 1.5 MB/s eta 0:00:0000:0100:09
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.9 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%pip install scikit-learn --break-system-packages


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 1.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 3.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import json

img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_data = datagen.flow_from_directory("dataset", target_size=img_size, subset='training', class_mode='categorical')
val_data = datagen.flow_from_directory("dataset", target_size=img_size, subset='validation', class_mode='categorical')

# Automatically determine number of classes
num_classes = len(train_data.class_indices)

# Save class labels
with open('class_names.json', 'w') as f:
    json.dump(train_data.class_indices, f)

# Handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights_dict = dict(enumerate(class_weights))

# Build the model
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, validation_data=val_data, epochs=10, class_weight=class_weights_dict)

# Save the model
model.save('plant_health_model.h5')


2025-03-26 17:38:57.812015: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-26 17:38:57.812375: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-26 17:38:57.814729: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-26 17:38:57.821418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742985537.833078  124526 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742985537.83

Found 5128 images belonging to 3 classes.
Found 1281 images belonging to 3 classes.


E0000 00:00:1742985539.349038  124526 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1742985539.349304  124526 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10


/home/peeranat/.local/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 55s 331ms/step - accuracy: 0.9002 - loss: 0.2356 - val_accuracy: 1.0000 - val_loss: 1.2059e-04
Epoch 2/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 53s 328ms/step - accuracy: 1.0000 - loss: 1.2189e-04 - val_accuracy: 1.0000 - val_loss: 1.0664e-04
Epoch 3/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 52s 323ms/step - accuracy: 1.0000 - loss: 1.0862e-04 - val_accuracy: 1.0000 - val_loss: 9.2484e-05
Epoch 4/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 51s 315ms/step - accuracy: 1.0000 - loss: 9.3051e-05 - val_accuracy: 1.0000 - val_loss: 7.9418e-05
Epoch 5/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 53s 329ms/step - accuracy: 1.0000 - loss: 7.5313e-05 - val_accuracy: 1.0000 - val_loss: 6.7807e-05
Epoch 6/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 53s 331ms/step - accuracy: 1.0000 - loss: 5.8981e-05 - val_accuracy: 1.0000 - val_loss: 5.8244e-05
Epoch 7/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 53s 328ms/step - accuracy: 1.0000 - loss: 5.0031e-05 - val_accuracy: 1.0000 - val_loss: 5.0372e-05
Epoch 8/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 54

In [3]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('plant_health_model.h5')

# Your class names
class_names = ['Healthy', 'Late_blight', 'Leaf_Mold','Early_blight','Leaf_spot']

# Load and preprocess test image
img_path = '/home/peeranat/lilia/late_blight.jpeg'  # 👈 Replace with your image file
img = cv2.imread(img_path)
img_resized = cv2.resize(img, (224, 224))  # Must match training input size
img_array = np.expand_dims(img_resized / 255.0, axis=0)

# Predict
prediction = model.predict(img_array)
predicted_class = class_names[np.argmax(prediction)]
confidence = np.max(prediction) * 100

# Print result
print(f"Prediction: {predicted_class} ({confidence:.2f}%)")

# Optional: Show the image with prediction
cv2.putText(img, f"{predicted_class} ({confidence:.2f}%)", (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
cv2.imshow("Prediction", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
Prediction: Healthy (100.00%)
